# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [58]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
# Create a path to the csv and read it into a Pandas DataFrame
cities_data = "../output_data/cities.csv"
cities_df = pd.read_csv(cities_data)
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port Elizabeth,-33.92,25.57,60.80,100,81,4.70,ZA,1605733606
1,1,Tilichiki,60.47,166.10,2.48,93,100,4.36,RU,1605733606
2,2,Saint-Joseph,-21.37,55.62,66.02,69,75,17.22,RE,1605733606
3,3,Faanui,-16.48,-151.75,79.72,78,100,13.91,PF,1605733607
4,4,Avarua,-21.21,-159.78,78.80,83,100,11.41,CK,1605733590
...,...,...,...,...,...,...,...,...,...,...
550,550,Zambezi,-13.54,23.10,68.50,87,97,3.15,ZM,1605733790
551,551,Tateyama,34.98,139.87,60.80,82,75,5.82,JP,1605733791
552,552,Kanjiža,46.07,20.05,43.86,88,0,5.03,RS,1605733791
553,553,Nova Olinda do Norte,-3.89,-59.09,81.79,77,100,1.99,BR,1605733792


In [23]:
# Not sure why there is a unnamed:0
del cities_df["Unnamed: 0"] 

In [24]:
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Elizabeth,-33.92,25.57,60.80,100,81,4.70,ZA,1605733606
1,Tilichiki,60.47,166.10,2.48,93,100,4.36,RU,1605733606
2,Saint-Joseph,-21.37,55.62,66.02,69,75,17.22,RE,1605733606
3,Faanui,-16.48,-151.75,79.72,78,100,13.91,PF,1605733607
4,Avarua,-21.21,-159.78,78.80,83,100,11.41,CK,1605733590


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

In [26]:
cities_df["Humidity"].max()

100

In [33]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=cities_df["Humidity"], max_intensity=100, 
                                 dissipating=False, point_radius = 3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
ideal_max_temp = cities_df.loc[(cities_df["Max Temp"] > 65) & (cities_df["Max Temp"] < 77)]
ideal_max_temp

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,Saint-Joseph,-21.37,55.62,66.02,69,75,17.22,RE,1605733606
17,Clarence Town,-32.58,151.78,70.03,72,0,8.05,AU,1605733611
22,Rikitea,-23.12,-134.97,74.57,75,100,5.61,PF,1605733613
33,Kualakapuas,-3.01,114.39,73.40,100,75,1.12,ID,1605733616
40,Arraial do Cabo,-22.97,-42.03,71.60,100,90,6.93,BR,1605733461
...,...,...,...,...,...,...,...,...,...
542,Duncan,34.50,-97.96,72.90,27,1,21.92,US,1605733788
546,Bilma,18.69,12.92,65.07,28,0,4.41,NE,1605733789
547,Chimoré,-16.98,-65.13,70.23,95,100,4.45,BO,1605733789
550,Zambezi,-13.54,23.10,68.50,87,97,3.15,ZM,1605733790


In [38]:
ideal_temp_hum = ideal_max_temp.loc[(ideal_max_temp["Humidity"] > 31) & (ideal_max_temp["Humidity"] < 51)]
ideal_temp_hum

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
90,Port Lincoln,-34.73,135.87,71.78,50,0,19.62,AU,1605733630
139,Porbandar,21.64,69.61,72.70,43,100,10.11,IN,1605733643
177,Kenitra,34.26,-6.58,66.20,45,0,4.70,MA,1605733655
201,Ancud,-41.87,-73.82,69.08,43,0,6.93,CL,1605733661
226,Dabola,10.75,-11.12,74.59,37,95,2.33,GN,1605733667
283,Karratha,-20.74,116.85,74.07,46,29,3.18,AU,1605733682
315,Louis Trichardt,-23.04,29.90,72.66,46,0,1.36,ZA,1605733691
331,Iberia,29.95,-91.75,69.55,45,1,8.05,US,1605733697
351,La Plata,-34.92,-57.95,74.89,47,98,1.99,AR,1605733631
370,Guerrero Negro,27.98,-114.06,76.93,47,0,13.20,MX,1605733719


In [39]:
ideal_temp_hum_wind = ideal_temp_hum.loc[(ideal_temp_hum["Wind Speed"] > 0) & (ideal_temp_hum["Wind Speed"] < 7)]
ideal_temp_hum_wind

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
177,Kenitra,34.26,-6.58,66.20,45,0,4.70,MA,1605733655
201,Ancud,-41.87,-73.82,69.08,43,0,6.93,CL,1605733661
226,Dabola,10.75,-11.12,74.59,37,95,2.33,GN,1605733667
283,Karratha,-20.74,116.85,74.07,46,29,3.18,AU,1605733682
315,Louis Trichardt,-23.04,29.90,72.66,46,0,1.36,ZA,1605733691
351,La Plata,-34.92,-57.95,74.89,47,98,1.99,AR,1605733631
405,Ankpa,7.37,7.63,76.66,37,0,2.55,NG,1605733736
417,Zvishavane,-20.33,30.07,75.02,37,11,0.11,ZW,1605733742
426,Shurugwi,-19.67,30.01,69.64,39,40,1.86,ZW,1605733744
466,Kisi,9.08,3.85,75.09,34,0,2.46,NG,1605733756


In [40]:
ideal_temp_hum_wind_cloud = ideal_temp_hum_wind.loc[ideal_temp_hum_wind["Cloudiness"] < 50]
ideal_weather = ideal_temp_hum_wind_cloud.reset_index()
del ideal_weather["index"]
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kenitra,34.26,-6.58,66.20,45,0,4.70,MA,1605733655
1,Ancud,-41.87,-73.82,69.08,43,0,6.93,CL,1605733661
2,Karratha,-20.74,116.85,74.07,46,29,3.18,AU,1605733682
3,Louis Trichardt,-23.04,29.90,72.66,46,0,1.36,ZA,1605733691
4,Ankpa,7.37,7.63,76.66,37,0,2.55,NG,1605733736
5,Zvishavane,-20.33,30.07,75.02,37,11,0.11,ZW,1605733742
6,Shurugwi,-19.67,30.01,69.64,39,40,1.86,ZW,1605733744
7,Kisi,9.08,3.85,75.09,34,0,2.46,NG,1605733756


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [78]:
hotel_df = ideal_weather
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kenitra,34.26,-6.58,66.20,45,0,4.70,MA,1605733655,
1,Ancud,-41.87,-73.82,69.08,43,0,6.93,CL,1605733661,
2,Karratha,-20.74,116.85,74.07,46,29,3.18,AU,1605733682,
3,Louis Trichardt,-23.04,29.90,72.66,46,0,1.36,ZA,1605733691,
4,Ankpa,7.37,7.63,76.66,37,0,2.55,NG,1605733736,
5,Zvishavane,-20.33,30.07,75.02,37,11,0.11,ZW,1605733742,
6,Shurugwi,-19.67,30.01,69.64,39,40,1.86,ZW,1605733744,
7,Kisi,9.08,3.85,75.09,34,0,2.46,NG,1605733756,


In [79]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    if row["Hotel Name"]=="":

        # change location each iteration while leaving original params in place
        params["location"] = f"{lat},{lng}"

        # make request and print url
        name = requests.get(base_url, params=params).json()
        

        hotel_df.loc[index, "Hotel Name"] = name["results"][0]["name"]


In [80]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kenitra,34.26,-6.58,66.20,45,0,4.70,MA,1605733655,Bienvenue chez vous
1,Ancud,-41.87,-73.82,69.08,43,0,6.93,CL,1605733661,Hotel Arena Gruesa
2,Karratha,-20.74,116.85,74.07,46,29,3.18,AU,1605733682,ibis Styles Karratha
3,Louis Trichardt,-23.04,29.90,72.66,46,0,1.36,ZA,1605733691,139 on Munnik Guest House
4,Ankpa,7.37,7.63,76.66,37,0,2.55,NG,1605733736,AA Suites
5,Zvishavane,-20.33,30.07,75.02,37,11,0.11,ZW,1605733742,Hotel Boutique Runde
6,Shurugwi,-19.67,30.01,69.64,39,40,1.86,ZW,1605733744,Flame Lily Lodge
7,Kisi,9.08,3.85,75.09,34,0,2.46,NG,1605733756,AMARC GUEST HOUSE


In [81]:
# Removing unneeded columns (temperature related and date)
del hotel_df["Max Temp"]
del hotel_df["Humidity"]
del hotel_df["Cloudiness"]
del hotel_df["Wind Speed"]
del hotel_df["Date"]

In [82]:
hotel_df

,City,Lat,Lng,Country,Hotel Name
0,Kenitra,34.26,-6.58,MA,Bienvenue chez vous
1,Ancud,-41.87,-73.82,CL,Hotel Arena Gruesa
2,Karratha,-20.74,116.85,AU,ibis Styles Karratha
3,Louis Trichardt,-23.04,29.90,ZA,139 on Munnik Guest House
4,Ankpa,7.37,7.63,NG,AA Suites
5,Zvishavane,-20.33,30.07,ZW,Hotel Boutique Runde
6,Shurugwi,-19.67,30.01,ZW,Flame Lily Lodge
7,Kisi,9.08,3.85,NG,AMARC GUEST HOUSE


In [83]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [87]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(locations,info_box_content= hotel_info)

fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))